# LMRv2.x validation

In [ ]:
%load_ext autoreload
%autoreload 2

import cfr
import numpy as np
print(cfr.__version__)
import pens
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

## Load Recon Job

Plot tas climate field reconstruction (mean over ensembles) and time series 

In [ ]:
res = cfr.ReconRes('./recons/lmr_reproduce_pda_v3/')
res.load(['tas', 'tas_gm'])

## Validate GMST 

Firstly against LMR offline

### Load LMR offline (original LMRv2.1)
We can do this using Pangeo-Forge. We then need to take the average over lat/lon in order to get the global mean surface temperature. We can then turn this into an Ensemble Time Series for more convenient plotting and comparison. 

In [ ]:
lmr_off = xr.open_dataset('./prev_data/gmt_MCruns_ensemble_full_LMRv2.1.nc')
lmr_off


In [ ]:
ens_values = lmr_off.gmt.stack(ensemble=['MCrun', 'members']).values  # Shape: (time, MCrun*members)
time_values = lmr_off.time.values
years = np.array([t.year for t in time_values])

In [ ]:
lmr_ens = cfr.EnsTS(time=years, value=ens_values, value_name='Temperature Anomaly (°C)')
lmr_ens.plot_qs()

### Load GMST from our Recon Job 

In [ ]:
res_ts = res.recons['tas_gm']

### Plot both versions of LMR to see comparison


In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

res_ts.plot_qs(ax=ax2, ylim=[-0.8, 0.6])
ax2.set_title('LMR Reproduced CFR Global Mean Temperature')

lmr_ens.plot_qs(ax=ax1, ylim=[-0.8, 0.6]) 
ax1.set_title('LMRv2.1 Global Mean Temperature')

plt.tight_layout()
plt.show()

### Comparison using PENS

In [ ]:
import pens
import seaborn as sns

plt.style.use('default')
pens.set_style()

In [ ]:
# Convert cfr EnsTS to pens EnsTS
glob_pens = pens.EnsembleTS(time=lmr_ens.time, value=lmr_ens.value)
glob_pens.label = 'Original LMR'
glob_pens.time_unit = 'years'
glob_pens.value_name = 'GMST'
glob_pens.value_unit = '\N{DEGREE SIGN}C'

res_pens = pens.EnsembleTS(time=res_ts.time, value=res_ts.value)
res_pens.label = 'Reproduced LMR'
res_pens.time_unit = 'years'
res_pens.value_name = 'GMST'
res_pens.value_unit = '\N{DEGREE SIGN}C'

In [ ]:
# Align time dimension for both EnsTS

glob_time = lmr_ens.time
res_time = res_ts.time.values

common_start = max(glob_time.min(), res_time.min())
common_end = min(glob_time.max(), res_time.max())

# Create time range array
timespan = np.array([common_start, common_end])

# Slice to common period
glob_pens_aligned = glob_pens.slice(timespan)
res_pens_aligned = res_pens.slice(timespan)

In [ ]:
orig_intra = glob_pens_aligned.distance()
repro_intra = res_pens_aligned.distance()

# Calculate inter-ensemble distance 
inter_dist = glob_pens_aligned.distance(res_pens_aligned.value)
    
# Calculate plume distance 
plume_dist = glob_pens_aligned.plume_distance(res_pens_aligned.value, max_dist=1.0)


In [ ]:
print("\nDistances between ensembles:")
print(f"Original intra-ensemble distance : {orig_intra},\ len={len(orig_intra)}", )
print(f"Reproduced intra-ensemble distance: {repro_intra},\ len={len(repro_intra)}")
print(f"Inter-ensemble distance: {inter_dist}")
print(f"Plume distance: {plume_dist}")

In [ ]:
# Create figure and plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot KDE for individual ensembles with explicit labels
sns.kdeplot(data=orig_intra, fill=False, ax=ax, common_norm=False, label='Original LMRv2.1')
sns.kdeplot(data=repro_intra, fill=False, ax=ax, common_norm=False, label='Reproduced LMRv2.1')

# Add inter-ensemble distribution
sns.kdeplot(data=inter_dist, fill=True, ax=ax, common_norm=False, color='silver', 
            label='inter-ensemble')

# Add plume distance line
ax.axvline(x=plume_dist, color="black", linestyle="--", label='plume distance')

# Add labels
ax.set_xlabel('Distance')
ax.set_ylabel('Density')
ax.set_title('Distance Distributions')
ax.legend()

plt.tight_layout()
plt.show()

### Load HadCRUT4

load dataset using xarray, then convert to a cfr Ensemble Time Series object

In [ ]:
had = xr.load_dataset('./analyses/HadCRUT/HadCRUT.4.4.0.0.median.nc')
had.head()

In [ ]:
gm = had.temperature_anomaly.mean(dim=['latitude', 'longitude'])
had_annual = gm.groupby('time.year').mean()

# Make sure values are properly shaped ([,:1])
had_values = had_annual.values
if had_values.ndim == 1:
    had_values = had_values[:, np.newaxis] 

# annualize HadCRUT4
had_ts_annual = cfr.EnsTS(
    time=had_annual.year.values,
    value=had_values,
    value_name='Temperature Anomaly'
)

# Compare LMR with annual data
had_ts_compared = res_ts.compare(
    had_ts_annual, 
    ref_name='HadCRUT4', 
    timespan=(1880, 2000)
)

fig, ax = had_ts_compared.plot_qs(figsize=[12, 4], xlim=[1850,2000])

### Load GISTEMP

In [ ]:
gis = xr.load_dataset('./analyses/GISTEMP/gistemp1200_ERSSTv4.nc')
gis.head()

In [ ]:
gm = gis.tempanomaly.mean(dim=['lat', 'lon'])
gis_annual = gm.groupby('time.year').mean()

# Make sure values are properly shaped ([,:1])
gis_values = gis_annual.values
if gis_values.ndim == 1:
    gis_values = gis_values[:, np.newaxis] 

# annualize GISTEMP
gis_ts_annual = cfr.EnsTS(
    time=gis_annual.year.values,
    value=gis_values,
    value_name='Temperature Anomaly'
)

# Compare LMR with annual data
gis_ts_compared = res_ts.compare(
    gis_ts_annual, 
    ref_name='GISTEMPv4', 
    timespan=(1880, 2000)
)

fig, ax = gis_ts_compared.plot_qs(figsize=[12, 4], xlim=[1880,2000])

### Load Berkeley Earth Surface Temperature

In [ ]:
best = xr.load_dataset('./analyses/BerkeleyEarth/Land_and_Ocean_LatLong1.nc')
best

In [ ]:

gm = best.temperature.mean(dim=['latitude', 'longitude'])

# Convert time coordinate to datetime index (specific to BEST)
time_index = pd.date_range(start='1850-01-01', periods=len(best.time), freq='M')
gm = gm.assign_coords(time=time_index)

best_annual = gm.groupby('time.year').mean()

# Reshape values if needed
best_values = best_annual.values
if best_values.ndim == 1:
    best_values = best_values[:, np.newaxis]

# annualize
best_ts_annual = cfr.EnsTS(
    time=best_annual.year.values,
    value=best_values,
    value_name='Temperature Anomaly'
)

# Compare LMR with annual data
best_ts_compared = res_ts.compare(
    best_ts_annual,
    ref_name='BEST',
    timespan=(1880, 2000)
)

fig, ax = best_ts_compared.plot_qs(figsize=[12, 4], xlim=[1850, 2000])

### Plot all together 

In [ ]:
# combine all datasets to calculate mean
all_refs = np.array([
   had_ts_compared.ref_value[:121],
   gis_ts_compared.ref_value[:121],
   best_ts_compared.ref_value[:121]
])
mean_ref = np.mean(all_refs, axis=0)

mean_ts_annual = cfr.EnsTS(
    time=best_annual.year.values[30:-14], #manually slice dates
    value=mean_ref,
    value_name='Temperature Anomaly'
)

mean_ts_compared = res_ts.compare(
    mean_ts_annual,
    ref_name='Consensus',
    timespan=(1880, 2000)
)

fig, ax = mean_ts_compared.plot_qs(figsize=[12, 4], xlim=[1850, 2000])

In [ ]:
# Create base plot without validation plot
fig, ax = had_ts_compared.plot_qs(
   figsize=[16, 8], 
   xlim=(1880, 2000),
   color='indianred',
   plot_valid=False 
)

for compared, color, label in [
   (had_ts_compared, 'black', 'HadCRUT5'),
   (gis_ts_compared, 'blue', 'GISTEMP'),
   (best_ts_compared, 'green', 'Berkeley Earth'),
   (mean_ts_compared, 'purple', 'Consensus')
   
]:
   stats = compared.valid_stats
   label = f'{label} (r={stats["corr"]:.2f}, CE={stats["CE"]:.2f})'
   ax.plot(compared.ref_time, compared.ref_value[:121], color=color, label=label)


plt.legend()
plt.title('LMRv2.x vs Observational Data 1880-2000')

## Validate tas climate field

### Load field reconstruction 

In [ ]:
lala = xr.open_dataset('./recons/air_MCruns_ensemble_mean_LMRv2.1.nc')
lala

In [ ]:
import cfr
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# Load the data as a ClimateField object
lmr_tas = cfr.ClimateField().load_nc(
    path='./recons/air_MCruns_ensemble_mean_LMRv2.1.nc',
    vn='air', 
    time_name='time',  
    lat_name='lat', 
    lon_name='lon'
)

# Take the mean across the MCrun dimension
if 'MCrun' in lmr_tas.da.dims:
    lmr_tas.da = lmr_tas.da.mean(dim='MCrun')

# Calculate the time mean across the entire dataset
lmr_tas_mean = lmr_tas.da.mean(dim='time')

# Create a new ClimateField with this time-mean data
mean_field = cfr.ClimateField()
mean_field.da = lmr_tas_mean
# Add a name to the DataArray to avoid issues
mean_field.da.name = 'air'

# Explicitly provide a title to avoid the t_value error
fig, ax = mean_field.plot(title="LMR 2m Air Temperature - Full Period Mean")

# Manually adjust the colormap and limits if needed
imgs = ax.get_images()
if len(imgs) > 0:
    im = imgs[0]
    im.set_cmap('RdBu_r')
    im.set_clim(-1, 1)  # adjust based on your data range
    plt.colorbar(im, ax=ax, label='Temperature Anomaly (°C)')

plt.tight_layout()
plt.show()



### Loading reproduced

In [ ]:
xr.open_dataset('./recons/lmr_reproduce_pda/job_r01_recon.nc')

In [ ]:
res_tas = res.recons['tas']
res_tas.plot()

### Load GISTEMP temperature field 

The Tardif et al. paper uses BEST for field validation, but it did not work with cfr for some reason. GISTEMP had the highest correlation for time series validation, and hence I decided to try it for the field validation as well. 

In [ ]:
target = cfr.ClimateField().load_nc(
    path='./analyses/GISTEMP/gistemp1200_ERSSTv5.nc',
    vn='tempanomaly', 
    time_name='time',  
    lat_name='lat', 
    lon_name='lon',
    use_cftime=True).get_anom((1951,1980))

In [ ]:
target = target.annualize(months=[12, 1, 2])
target.da

#### Correlation

In [ ]:
valid_fd = res.recons['tas'].compare(
    target, stat='corr',
    timespan=(1880, 2000),
)

In [ ]:
valid_fd.plot_kwargs.update({'cbar_orientation': 'horizontal', 'cbar_pad': 0.1})

fig, ax = valid_fd.plot(
    title=f'corr(recon, obs), mean={valid_fd.geo_mean().value[0,0]:.2f}',
    projection='PlateCarree',
    latlon_range=(-90, 90, 0, 360),
    plot_cbar=True, plot_proxydb=False,
)

cfr.showfig(fig)

#### CE (coefficient of efficiency)

In [ ]:
valid_fdx = res.recons['tas'].compare(
    target, stat='CE',
    timespan=(1880, 2000),
)

In [ ]:
valid_fdx.plot_kwargs.update({'cbar_orientation': 'horizontal', 'cbar_pad': 0.1})

fig, ax = valid_fdx.plot(
    title=f'CE(recon, obs), mean={valid_fdx.geo_mean().value[0,0]:.2f}',
    projection='PlateCarree',
    latlon_range=(-90, 90, 0, 360),
    plot_cbar=True, plot_proxydb=False,
)

cfr.showfig(fig)

## Trying BEST again
Same process as above, but wrapping the longitude and alterating time array to be in datetime format.

In [ ]:
ds = xr.open_dataset('./analyses/BerkeleyEarth/Land_and_Ocean_LatLong1.nc')

# 2. Convert decimal years to datetime
years = ds.time.values
dates = pd.to_datetime([f"{int(year)}-{int((year % 1) * 12 + 1):02d}-15" for year in years])

# 3. Create a new DataArray with proper coordinates
da = xr.DataArray(
    ds.temperature.values,
    coords={
        'time': dates,
        'lat': ds.latitude.values,
        'lon': ds.longitude.values
    },
    dims=['time', 'lat', 'lon'],
    name='temperature'
)

# 4. Now create ClimateField object
target_be = cfr.ClimateField(da).get_anom(ref_period=[1951, 1980])
target_be = target.annualize(months=[12, 1, 2])
target_be.da

In [ ]:
# Convert target data to 0-360 format first
target_wrapped = target_be.wrap_lon(mode='360')

# Then do the comparison
valid_fdb = res.recons['tas'].compare(
    target_wrapped,
    stat='corr',
    timespan=(1880, 2000),
)

# Plot with updated settings
valid_fdb.plot_kwargs.update({'cbar_orientation': 'horizontal', 'cbar_pad': 0.1})
fig, ax = valid_fdb.plot(
    title=f'CFR Reproduced corr(recon, obs), mean={valid_fdb.geo_mean().value[0,0]:.2f}',
    projection='PlateCarree',
    latlon_range=(-90, 90, 0, 360),
    plot_cbar=True,
    plot_proxydb=False,
    cmap='bwr'
)

cfr.showfig(fig)

In [ ]:
# Then do the comparison
valid_fdbx = res.recons['tas'].compare(
    target_wrapped,
    stat='CE',
    timespan=(1880, 2000),
)

# Plot with updated settings
valid_fdbx.plot_kwargs.update({'cbar_orientation': 'horizontal', 'cbar_pad': 0.1})
fig, ax = valid_fdbx.plot(
    title=f'CFR reproduced CE(recon, obs), mean={valid_fdbx.geo_mean().value[0,0]:.2f}',
    projection='PlateCarree',
    latlon_range=(-90, 90, 0, 360),
    plot_cbar=True,
    plot_proxydb=False,
    cmap='bwr'
)

cfr.showfig(fig)

## valid with original lmr

In [ ]:
lmr_tas = cfr.ClimateField().load_nc(
    path='./recons/air_MCruns_ensemble_mean_LMRv2.1.nc',
    vn='air', 
    time_name='time',  
    lat_name='lat', 
    lon_name='lon'
)

target_lmr = lmr_tas.get_anom(ref_period=[1951, 1980])
filtered_da = target_lmr.da.sel(time=target_lmr.da.time.dt.year != 0)
target_lmr = cfr.ClimateField(da=filtered_da)
target_lmr = target_lmr.annualize(months=[12, 1, 2])

target_lmr_mean = cfr.ClimateField(da=target_lmr.da.mean(dim='MCrun'))


In [ ]:
print(target_lmr.da.dims)

In [ ]:
# Then do the comparison
valid_fdb = target_lmr_mean.compare(
    target_wrapped,
    stat='corr',
    timespan=(1880, 2000),
)

# Plot with updated settings
valid_fdb.plot_kwargs.update({'cbar_orientation': 'horizontal', 'cbar_pad': 0.1})
fig, ax = valid_fdb.plot(
    title=f'LMRv2.1 corr(recon, obs), mean={valid_fdb.geo_mean().value[0,0]:.2f}',
    projection='PlateCarree',
    latlon_range=(-90, 90, 0, 360),
    plot_cbar=True,
    plot_proxydb=False,
    cmap='bwr'
)

cfr.showfig(fig)

In [ ]:
# Then do the comparison
valid_fdbx = target_lmr_mean.compare(
    target_wrapped,
    stat='CE',
    timespan=(1880, 2000),
)

# Plot with updated settings
valid_fdbx.plot_kwargs.update({'cbar_orientation': 'horizontal', 'cbar_pad': 0.1})
fig, ax = valid_fdbx.plot(
    title=f'LMRv2.1 CE(recon, obs), mean={valid_fdbx.geo_mean().value[0,0]:.2f}',
    projection='PlateCarree',
    latlon_range=(-90, 90, 0, 360),
    plot_cbar=True,
    plot_proxydb=False,
    cmap='bwr'
)

cfr.showfig(fig)